# Случайные леса
__Суммарное количество баллов: 10__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

In [70]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
#from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
import random
import csv


### Задание 1 (3 балла)
Реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [65]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self.X = []
        self.y = []
        self.n_estimators = n_estimators
        self.trees = []
        
        for i in range(n_estimators):
            self.trees.append(DecisionTreeClassifier(max_depth=max_depth, 
                                                     min_samples_leaf=min_samples_leaf, 
                                                     max_features=max_features, 
                                                     criterion=criterion))  
    def fit(self, X, y):
        self.X = X
        self.y = y
        inds = []
        X_n = []
        y_n = []
        
        for i in range(len(X)):
            inds.append(random.randint(0, len(X)-1))
                
        for i in inds:
            X_n.append(X[i])
            y_n.append(y[i]) 

        X_np = np.array(X_n)
        y_np = np.array(y_n)
        for i in range(self.n_estimators):
            self.trees[i].fit(X_np, y_np)
        


    def predict(self, X):
        classes = []
        for i in range(self.n_estimators):
            for j in range(len(X)):
                classes.append(self.trees[i].predict([X[j]])[0])
        classes = np.array(classes)
        classes.shape = ((self.n_estimators, len(X)))
        res = []
        for j in range(len(classes[0])):
            ones = 0
            zeros = 0
            for i in range(len(classes)):
                if classes[i][j] == 1:
                    ones += 1
                else:
                    zeros += 1
            if ones > zeros:
                res.append(1)
            else:
                res.append(0)
                
        return res



predicting...


[0, 1]

### Задание 3 (3 балла)
Оптимизируйте по AUC на кроссвалидации (размер валидационной выборки - 20%) параметры своей реализации Random Forest: 

максимальную глубину деревьев из [2, 3, 5, 7, 10], количество деревьев из [5, 10, 20, 30, 50, 100]. 

Постройте ROC кривую (и выведите AUC и accuracy) для лучшего варианта.

Подсказка: можно построить сразу 100 деревьев глубины 10, а потом убирать деревья и
глубину.

In [144]:
def read_cancer_dataset(path_to_csv):
    # Возвращает пару из X и y. X - массив векторов. y - соответствующие векторам метки
    f = open(path_to_csv, 'r')
    X = np.array([[]])
    y = np.array([[]])
    reader = csv.reader(f)
    for row in reader:
        list = [[]]
        label = 0
        for col in row:
            if col == 'label':
                break
            if col == 'M':
                label = 1
            if col != 'M' and col != 'B':
               list[0].append(float(col))

        if y[0].size == 0:
            y = np.array([label])
        else:
            y = np.append(y, [label])
        if X[0].size == 0:
            X = np.array(list)
        else:
            X = np.append(X, list, axis = 0)

    f.close()
    return X[:][:], y[1:]

def read_spam_dataset(path_to_csv):
    # Возвращает пару из X и y. X - массив векторов. y - соответствующие векторам метки
    f = open(path_to_csv, 'r')
    X = np.array([[]])
    y = np.array([[]])
    reader = csv.reader(f)
    k = 0
    for row in reader:
        if k == 0:
            k = 1
            continue
        list = [[]]
        for i in range(len(row)):
            if i != len(row)-1:
                list[0].append(float(row[i]))
            else:
                if y[0].size == 0:
                    y = np.array([int(row[i])])
                else:
                    y = np.append(y, [int(row[i])])

        if X[0].size == 0:
            X = np.array(list)
        else:
            X = np.append(X, list, axis = 0)

    f.close()
    return X[:][:], y[1:]

def train_test_split(X, y, ratio):
    # Возвращает X_train, y_train, X_test, y_test
    # X_train и X_test - массив векторов - две части массива X, разделенного в состветсви с коэффициентом ratio
    # y_train и y_test - соответствующие X_train и X_test метки классов
    X_train = np.array([[]])
    X_test = np.array([[]])
    y_train = np.array([])
    y_test = np.array([])
    trainlist = []
    testlist = []
    train = int(ratio*len(y))

    for i in range(train):
        tr = random.randint(0, len(y)-1)
        while tr in trainlist:
            tr = random.randint(0, len(y)-1)

        trainlist.append(tr)

    for i in range(0, len(y)):
        if i not in trainlist:
            testlist.append(i)

    X_train = X[trainlist][:]
    X_test = X[testlist][:]
    y_train = y[trainlist]
    y_test = y[testlist]

    return X_train, y_train, X_test, y_test

def get_accuracy(y_test, y_pred):
    TP = 0
    FP = 0
    FN = 0
    TN = 0
    for i in range(len(y_test)):
        if (y_test[i] == 1) and (y_pred[i] == 1):
            TP += 1
        elif (y_test[i] == 0) and (y_pred[i] == 1):
            FP += 1
        elif (y_test[i] == 1) and (y_pred[i] == 0):
            FN += 1
        else:
            TN += 1
    return (TP + TN) / (TP + TN + FP + FN)

def train_test_split_cross(X, y, r, ind):
    p = np.random.permutation(len(y))
    X = X[p]
    y = y[p]
    len_ind = []
    samples = int(1 / r)
    for i in range(samples):
        if i == samples-1:
            len_ind.append(len(y) - int(len(y)*r)*(samples-1))
        else:
            len_ind.append(int(len(y)*r))
    X_test = []
    y_test = []
    X_train = []
    y_train = []
    
    prev = 0
    if ind != 0:
        for i in range(ind-1):
            prev += len_ind[i]

    curr = 0
    for i in range(len(len_ind)):
        if i == 0:
            curr = 0
        elif i == 1:
            curr = len_ind[0] 
        else:
            curr = 0
            for j in range(0, i-1):
                curr += len_ind[i-1]
        for j in range(len_ind[i]):
            if ind != i:
                #print(i, curr)
                X_train.append(X[curr + j])
                y_train.append(y[curr + j])
            else:
                X_test.append(X[prev + j])
                y_test.append(y[prev + j])
        if ind == i:        
            y_test = np.array(y_test)
            X_test = np.array(X_test)
            X_test.shape = ((len_ind[i], len(X[0])))
            
    curr = 0
    for i in range(len(len_ind)):
        if i != ind:
            curr += len_ind[i]
    X_train = np.array(X_train)
    X_train.shape = ((curr), len(X[0]))
    y_train = np.array(y_train)
    return X_test, y_test, X_train, y_train
            
def optimise(file, r = 0.2):
    if file == 'cancer.csv':
        X, y = read_cancer_dataset(file)
    if file == 'spam.csv':
        X, y = read_spam_dataset(file)
        
    depths = [10, 7, 5, 3, 2]
    nums = [100, 50, 30, 20, 10, 5]
    opt_depth = 10
    opt_depth = 100

    res_auc = 0
    res_depth = 0
    res_num = 0
    res_accuracy = 0
    
    
    for d in depths:
        for n in nums:
            auc = 0
            res_accuracy = 0
            for i in range(int(1/r)):           
                X_test, y_test, X_train, y_train = train_test_split_cross(X, y, 0.2, i)
                rforest = RandomForestClassifier(max_depth=d, n_estimators=n)
                rforest.fit(X_train, y_train)
                y_pred = rforest.predict(X_test)
                auc += metrics.roc_auc_score(y_test, y_pred)
                res_accuracy += get_accuracy(y_test, y_pred)
            auc /= int(1/r)
            res_accuracy /= int(1/r)
            if res_auc < auc:
                res_auc = auc
                res_depth = d
                res_num = n
    print('number of tres: ', res_num,
          '\nnumber of depth: ', res_depth,
          '\nauc: ', res_auc,
          '\naccuracy: ', res_accuracy)
 
print('for cancer:')
optimise('cancer.csv')
print('\nfor spam:')
optimise('spam.csv')

for cancer:
number of tres:  100 
number of depth:  5 
auc:  0.9731265796185727 
accuracy:  0.9246804326450345

for spam:
number of tres:  50 
number of depth:  10 
auc:  0.9420806029612917 
accuracy:  0.8513043478260869


### Задание 4 (3 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest выглядит так:
1. Посчитать out-of-bag ошибку предсказания `err_oob` (https://en.wikipedia.org/wiki/Out-of-bag_error)
2. Перемешать значения признака `j` у объектов выборки (у каждого из объектов изменится значение признака `j` на какой-то другой)
3. Посчитать out-of-bag ошибку (`err_oob_j`) еще раз.
4. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [ ]:
def feature_importance(rfc):
    raise NotImplementedError()

def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Протестируйте решение на простом синтетическом наборе данных. В результате должна получиться точность `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [ ]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Проверьте, какие признаки важны для датасетов cancer и spam?

### Задание 5 (1 балл)
В качестве аьтернативы попробуем CatBoost. 

Туториалы можно найти, например, [здесь](https://catboost.ai/docs/) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb).

Также, как и реализованный ними RandomForest, примените его для наших датасетов.